In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer

In [2]:
data = pd.read_csv('train.tsv',sep='\t')
tags = ["part-time-job", "full-time-job", "hourly-wage", "salary", "associate-needed", "bs-degree-needed", "ms-or-phd-needed", "licence-needed", "1-year-experience-needed", "2-4-years-experience-needed", "5-plus-years-experience-needed", "supervising-job"]
data.head()

,tags,description
0,licence-needed supervising-job 5-plus-years-ex...,THE COMPANY Employer is a midstream service...
1,2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
2,part-time-job,This is a great position for the right person....
3,licence-needed,A large multi-specialty health center is expan...
4,5-plus-years-experience-needed full-time-job b...,JOB PURPOSE: The Account Director is respon...


In [3]:
for index, row in data.iterrows():   
    row.description = row.description.lower()

In [4]:
y = []

for tag in tags:
    y_curr = []
    for tags_list in data.tags:
        if (type(tags_list) != float) and (tag in tags_list):
            y_curr.append(1)
        else:
            y_curr.append(0)
    y.append(y_curr)
y = np.array(y).T

The same score function as used at HackerRank contest

In [10]:
def score(predictions, true_values):
    if predictions.shape != true_values.shape:
        print 'Error: dimensions must be the same'
        return
    STP, SFP, STN, SFN = 0, 0, 0, 0
    for i in xrange(len(tags)):
        STP += sum([1 if true_values[j,i]==1 and predictions[j,i]==1 else 0 for j in xrange(len(predictions))])
        SFP += sum([1 if true_values[j,i]==0 and predictions[j,i]==1 else 0 for j in xrange(len(predictions))])
        STN += sum([1 if true_values[j,i]==0 and predictions[j,i]==0 else 0 for j in xrange(len(predictions))])
        SFN += sum([1 if true_values[j,i]==1 and predictions[j,i]==0 else 0 for j in xrange(len(predictions))])
    P = float(STP)/(STP+SFP)
    R = float(STP)/(STP+SFN)
    return P, R, 2*P*R/(P+R)

In [5]:
lookup = [["part time", "part-time"], ["full time", "full-time"], ["hourly", "hour"], 
          ["salary"], ["associate"], ["bachelor", " bs ", "/bs " " bs/"], ["master", "phd"], 
          ["licence", "license", "certif", "eligible"], ["1 year", "one year"], 
          ["2 years" , "3 years" ,"4 years", "two years" , "three years" ,"four years", "2+ years", "3+ years", "4+ years"], 
          ["5 years", "5+ years", "five years", "0 years"], ["supervi", "manager"]]

In [6]:
predictions = np.zeros_like(y)
for i,descrip in enumerate(data.description):
    for j,keywords in enumerate(lookup):
        for word in keywords:
            if word in descrip:
                predictions[i,j] = 1

In [7]:
print predictions[1]
print y[1]

[0 1 1 1 0 0 0 0 0 0 1 1]
[0 1 0 1 0 0 0 0 0 1 0 0]


In [8]:
data.description[1]

'icr staffing is now accepting resumes for industrial maintenance mechanics. this is an experienced position, requiring prior hands-on, working experience. successful candidates will possess a positive, high-energy and professional attitude with a strong desire to work and achieve. the position is located in the southern california, inland-empire, is full-time and offers a starting salary of 43 - 53k per year, d.o.e. this position is temp-to-hire, though for highly qualified candidates there is potential for direct hire to include benefits.  successful maintenance mechanics will possess\xe2\x80\xa6  \xe2\x80\xa6 3-5 years hands-on, manufacturing  or plastics industry experience.  \xe2\x80\xa6 knowledge and experience with; hydraulics, pneumatics, pumps, valves, cylinders, scales, conveyors.  \xe2\x80\xa6 electrical knowledge of; ac/dc motor controls and drives, control circuitry.   \xe2\x80\xa6 ability to read schematics, maintain electrical systems up to 480v 3-phase, run conduit and 

In [12]:
print 'recall %s precision %s f1_score %s' %score(predictions, y)

recall 0.489067656766 precision 0.689244186047 f1_score 0.572152509653
